# Coffe Can Momentum Strategy
This is an exploration of two coffee can strategies. The coffee can idea is where you only buy and hold for the duration of the investment period. No selling, shorting or rebalancing.
In this notebook, I apply this technique to two strategies:
- Buy the top performers of the last 6 months.
- Buy the worst performers of the last 6 months.

## Helper functions

In [1]:
from datetime import datetime
import pandas as pd

def filter_date(df):
    start_date = df.index[0]
    if start_date.month > 1:
        start_date = start_date.replace(month=6)

    if start_date.month > 6:
        start_date = start_date.replace(year=start_date.year+1)
        start_date = start_date.replace(month=1)
    df = df.iloc[df.index >= pd.to_datetime(start_date.strftime("%m-%d-%Y"))]
    df = df.iloc[(df.index.month == 6) | (df.index.month == 1)]
    return df

def get_change(df, step=1):
    df = filter_date(df)
    return (df.iloc[::step, :].pct_change(periods=1)*100).dropna()

## Prepare data over period

In [2]:
path = "./price_change"
benchmark_ticker = 'spy'

In [ ]:
from yahooquery import Ticker
import pandas as pd
import json

init_tickers = []
with open('./us_tickers.json', 'r') as f:
    init_tickers = json.load(f)

change_df = pd.DataFrame({})
price_df = pd.DataFrame({})

# Get benchmark (SPY)
ticker = Ticker(benchmark_ticker, asynchronous=True)
df = ticker.history(period='20y', interval='1mo', adj_ohlc=True, adj_timezone=False)
df = df.reset_index()
df['date'] = pd.to_datetime(df['date'], utc=True)
df['date'] = df['date'].dt.tz_localize(None)
close = df[['date','close']].set_index('date')
price_df = pd.concat([price_df, filter_date(close).rename(columns={"close": benchmark_ticker})], axis=1)

for idx, t in enumerate(init_tickers):
    try:
        ticker = Ticker(t, asynchronous=True)
        details = ticker.summary_detail[t]
        if 'marketCap' not in details or details['marketCap'] < 500000000:
            continue

        df = ticker.history(period='20y', interval='1mo', adj_ohlc=True, adj_timezone=False)
        df = df.reset_index()
        df['date'] = pd.to_datetime(df['date'], utc=True)
        df['date'] = df['date'].dt.tz_localize(None)
        close = df[['date','close']].set_index('date')

        price_df = pd.concat([price_df, filter_date(close).rename(columns={"close": t})], axis=1)
        change_df = pd.concat([change_df, get_change(close).rename(columns={"close": t})], axis=1)
        if idx % 100 == 0:
            change_df.to_csv("{}/change_tmp.csv".format(path), sep=';')
            price_df.to_csv("{}/price_tmp.csv".format(path), sep=';')
    except:
        pass

change_df.to_csv("{}/change.csv".format(path), sep=';')
price_df.to_csv("{}/price.csv".format(path), sep=';')
price_df.head()

## Load data

In [69]:
price_df = pd.read_csv("{}/price.csv".format(path), sep=';')
change_df = pd.read_csv("{}/change.csv".format(path), sep=';')
price_df.head()

,date,spy,A,AA,AAC,AAL,AAON,AAP,AAPL,AAT,...,ZIP,ZLAB,ZM,ZNTL,ZS,ZTO,ZTS,ZUO,ZWS,ZYME
0,2003-06-01,66.531387,11.928134,47.999783,NaN,NaN,3.047452,17.986330,0.289284,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2004-01-01,78.278030,22.489561,65.004814,NaN,NaN,3.133018,22.989416,0.342405,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2004-06-01,79.279274,17.864737,63.400837,NaN,NaN,3.322249,26.096426,0.493877,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2005-01-01,83.067123,13.490077,57.183121,NaN,NaN,2.382674,25.458477,1.167153,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2005-06-01,84.113289,14.045305,51.153660,NaN,NaN,2.922395,38.128647,1.117371,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Filter data

In [70]:
start_year = datetime(year=2004,month=1,day=1)
end_year = datetime(year=2023,month=1,day=1)

price_df = price_df[pd.to_datetime(price_df["date"]) >= start_year].dropna(axis=1, how='all')
price_df = price_df[pd.to_datetime(price_df["date"]) <= end_year].dropna(axis=1, how='all')

change_df = change_df[pd.to_datetime(change_df["date"]) >= start_year].dropna(axis=1, how='all')
change_df = change_df[pd.to_datetime(change_df["date"]) <= end_year].dropna(axis=1, how='all')

## Simple Momentum Coffee Can Algorithm
for each ticker:

    - compute change over period
    - store computed changes and prices

for each date:

    - get top N best movers over period
    - invest M money into the top N movers

In [71]:
import numpy as np
import math

# Algo params
periodic_alloc_amount = 6000
top_N = 20
portfolio = {}
benchmark = {}

# Create dict for tickers and their column index
col_idx = {}
for idx, c in enumerate(change_df.columns.to_numpy()[1:]):
    col_idx[idx] = c

# Algo
total_invested = 0
for _, data in enumerate(change_df.to_numpy()):
    date = data[0]
    arr = data[1:].astype(float)

    # Get top movers
    np.nan_to_num(arr, copy=False, nan=-np.inf)
    sorted_idx = np.argsort(arr)
    sorted_arr = arr[sorted_idx]
    top_movers = sorted_arr[-top_N : ]
    top_movers_idx = sorted_idx[-top_N : ]

    am = periodic_alloc_amount/top_N
    total_invested += periodic_alloc_amount
    # Get price data for top movers and buy them
    for t in top_movers_idx:
        price = price_df[price_df['date'] == date][col_idx[t]].values[0]
        shares = math.floor(am/price)
        #if date == '2004-01-01':
        #    print('invested {} in {}'.format(price*shares, col_idx[t]))
        if col_idx[t] not in portfolio:
            portfolio[col_idx[t]] = shares
        else:
            portfolio[col_idx[t]] = portfolio[col_idx[t]] + shares
    
    # Benchmark
    price = price_df[price_df['date'] == date][benchmark_ticker].values[0]
    shares = math.floor(periodic_alloc_amount/price)
    if benchmark_ticker not in benchmark:
        benchmark[benchmark_ticker] = shares
    else:
        benchmark[benchmark_ticker] = benchmark[benchmark_ticker] + shares
    

print(portfolio)
print(benchmark)
print(total_invested)

# portfolio valuation
valuation = 0
for stock, shares in portfolio.items():
    price = price_df.iloc[-1:][stock].values[0]
    valuation += price*shares

# benchmark valuation
benchmark_val = 0
price = price_df.iloc[-1:][benchmark_ticker].values[0]
benchmark_val += price*benchmark[benchmark_ticker]

print('valuation: {0:.0f}'.format(valuation))
print("performance: {0:.0f}%".format(((valuation/total_invested)-1)*100))
print('benchmark valuation: {0:.0f}'.format(benchmark_val))
print('benchmark performance: {0:.0f}%'.format(((benchmark_val/total_invested)-1)*100))
print('CAGR: {0:.0f}%'.format(((valuation/total_invested)**(1/20)-1)*100))
print('benchmark CAGR: {0:.0f}%'.format(((benchmark_val/total_invested)**(1/20)-1)*100))


{'HCKT': 55, 'DECK': 50, 'PRFT': 81, 'UHAL': 509, 'LNG': 88, 'CENX': 30, 'BAK': 36, 'TTMI': 17, 'CDE': 33, 'BB': 20, 'CMCO': 43, 'EQIX': 11, 'MGIC': 118, 'BBSI': 38, 'CAMT': 194, 'CALM': 82, 'ARWR': 120, 'SBRA': 18, 'EHC': 18, 'AXON': 75, 'SKX': 145, 'CME': 18, 'ADSK': 14, 'ABEV': 433, 'WCC': 16, 'GPK': 40, 'FNB': 34, 'ATI': 26, 'MT': 11, 'NSSC': 217, 'AMED': 12, 'OSTK': 42, 'HLIO': 128, 'NAT': 41, 'GOGL': 6, 'TWI': 37, 'MNST': 1550, 'DENN': 140, 'IIIN': 216, 'SEB': 0, 'MCRI': 19, 'SIRI': 785, 'BRFS': 102, 'ADEA': 77, 'CIB': 41, 'TKC': 50, 'FRO': 5, 'SIM': 44, 'CLF': 80, 'AAPL': 257, 'SPTN': 81, 'VALE': 80, 'OSPN': 75, 'VCEL': 2, 'WT': 210, 'NSP': 37, 'VRTX': 17, 'CNQ': 26, 'BLFS': 458, 'SQM': 64, 'GME': 375, 'SCVL': 48, 'TELL': 57, 'ATRO': 242, 'PFBC': 2, 'SWN': 25, 'ITRI': 6, 'ESTE': 37, 'SWBI': 222, 'NDAQ': 78, 'PAR': 30, 'FC': 39, 'GGB': 87, 'WIRE': 11, 'STRL': 16, 'AGI': 77, 'TIMB': 11, 'AMD': 7, 'ASGN': 24, 'GES': 24, 'INSM': 61, 'HOLX': 11, 'ISRG': 29, 'RES': 38, 'NVAX': 8, 'REP

## Simple Undervalued Coffee Can Algorithm (BTFD and HODL)
for each ticker:

    - compute change over period
    - store computed changes and prices

for each date:

    - get top N worse movers over period
    - invest M money into the bottom N movers

In [72]:
import numpy as np
import math

# Algo params
periodic_alloc_amount = 6000
bottom_N = 20
portfolio = {}
benchmark = {}

# Create dict for tickers and their column index
col_idx = {}
for idx, c in enumerate(change_df.columns.to_numpy()[1:]):
    col_idx[idx] = c

# Algo
total_invested = 0
for _, data in enumerate(change_df.to_numpy()):
    date = data[0]
    arr = data[1:].astype(float)

    # Get worse movers
    np.nan_to_num(arr, copy=False, nan=np.inf)
    sorted_idx = np.argsort(arr)
    sorted_arr = arr[sorted_idx]
    worse_movers = sorted_arr[:bottom_N]
    worse_movers_idx = sorted_idx[:bottom_N]

    am = periodic_alloc_amount/bottom_N
    total_invested += periodic_alloc_amount
    # Get price data for top movers and buy them
    for t in worse_movers_idx:
        price = price_df[price_df['date'] == date][col_idx[t]].values[0]
        shares = math.floor(am/price)
        #if date == '2008-01-01':
        #    print('invested {} in {}'.format(price*shares, col_idx[t]))
        if col_idx[t] not in portfolio:
            portfolio[col_idx[t]] = shares
        else:
            portfolio[col_idx[t]] = portfolio[col_idx[t]] + shares
    
    # Benchmark
    price = price_df[price_df['date'] == date][benchmark_ticker].values[0]
    shares = math.floor(periodic_alloc_amount/price)
    if benchmark_ticker not in benchmark:
        benchmark[benchmark_ticker] = shares
    else:
        benchmark[benchmark_ticker] = benchmark[benchmark_ticker] + shares
    

print(portfolio)
print(benchmark)
print(total_invested)

# portfolio valuation
valuation = 0
for stock, shares in portfolio.items():
    price = price_df.iloc[-1:][stock].values[0]
    valuation += price*shares

# benchmark valuation
benchmark_val = 0
price = price_df.iloc[-1:][benchmark_ticker].values[0]
benchmark_val += price*benchmark[benchmark_ticker]

print('valuation: {0:.0f}'.format(valuation))
print("performance: {0:.0f}%".format(((valuation/total_invested)-1)*100))
print('benchmark valuation: {0:.0f}'.format(benchmark_val))
print('benchmark performance: {0:.0f}%'.format(((benchmark_val/total_invested)-1)*100))
print('CAGR: {0:.0f}%'.format(((valuation/total_invested)**(1/20)-1)*100))
print('benchmark CAGR: {0:.0f}%'.format(((benchmark_val/total_invested)**(1/20)-1)*100))


{'BHC': 34, 'PSMT': 58, 'FCN': 20, 'WT': 5999, 'RGEN': 278, 'SRPT': 60, 'VRTX': 30, 'AORT': 110, 'PTSI': 158, 'FNB': 65, 'EXAS': 173, 'DENN': 576, 'LUMN': 69, 'STAA': 162, 'RYAAY': 20, 'BLFS': 1374, 'ABC': 28, 'NSSC': 528, 'CLH': 78, 'DMRC': 24, 'CCOI': 152, 'DSGX': 272, 'REPX': 54, 'MTZ': 55, 'HSKA': 75, 'AMKR': 106, 'QDEL': 50, 'RDNT': 586, 'CIEN': 11, 'NVMI': 75, 'RDWR': 35, 'IMOS': 528, 'SPNS': 140, 'VCEL': 130, 'MED': 105, 'RMBS': 16, 'RDY': 39, 'BBAR': 311, 'XPO': 513, 'NFLX': 182, 'TSEM': 10, 'MGPI': 651, 'NVAX': 106, 'AEIS': 42, 'ALGN': 34, 'OPK': 833, 'IESC': 64, 'DRD': 234, 'ALE': 14, 'ADTN': 23, 'ARWR': 441, 'VIAV': 30, 'NOVT': 119, 'OMCL': 40, 'SNPS': 17, 'CTIC': 374, 'BAK': 32, 'HLIT': 62, 'CALM': 149, 'TREX': 383, 'BIIB': 8, 'GPK': 95, 'OFG': 30, 'BLU': 677, 'INSM': 141, 'AXON': 29, 'SIM': 78, 'SYNA': 21, 'MGIC': 246, 'AMOT': 109, 'AGX': 182, 'TARO': 23, 'TPX': 158, 'PRMW': 80, 'JBSS': 36, 'THC': 10, 'MOH': 17, 'TTWO': 18, 'MVIS': 757, 'NEO': 1428, 'NYMT': 23, 'PBH': 24, 

## Conclusion
Both strategies beat the benchmark during various periods within 2004 and 2023. Buying the worst performing stock however yield a much better result than buying the top performers.